In [1]:
# импортируем необходимые библиотеки и
# функцию haversine() пакета haversine
import pandas as pd 
import numpy as np
from haversine import haversine
# загружаем и смотрим данные
data = pd.read_csv('Data/Flats.csv', encoding='cp1251', sep=';')
data.head()

,Rooms_Number,Object_Type,Settlement,District,Street,House_Number,Metro,Metro_m,Space_Total,Lat,Long,Date_Postr
0,1,Хрущевка,Новосибирск,Первомайский,Маяковского,24,Речной вокзал,11260,30.5,54.9738,83.0860,1971
1,1,Типовая,Новосибирск,Первомайский,Звездная,14,Речной вокзал,12130,28.9,54.9551,83.0681,1974
2,1,Типовая,Новосибирск,Центральный,Лермонтова,36,Сибирская,438,31.9,55.0446,82.9267,1973
3,2,Хрущевка,Новосибирск,Центральный,Достоевского,20,Сибирская,312,44.7,55.0456,82.9192,1964
4,2,Улучшенной планировки,Новосибирск,Советский,Демакова,12,Речной вокзал,23340,49.0,54.8634,83.1041,1989


In [2]:
# пишем функцию, вычисляющую расстояние по формуле гаверсинусов
def calculate_distance(lat1, long1, lat2, long2, km=False):
    rad = 6372795
    
    if km:
        rad = np.round(rad, -3) / 1000
    
    lat1 = lat1 * np.pi / 180.
    long1 = long1 * np.pi / 180.
    lat2 = lat2 * np.pi / 180.
    long2 = long2 * np.pi / 180.
    
    cl1 = np.cos(lat1)
    cl2 = np.cos(lat2)
    sl1 = np.sin(lat1)
    sl2 = np.sin(lat2)
    delta = long2 - long1
    cdelta = np.cos(delta)
    sdelta = np.sin(delta)
    
    y = np.power(np.power(cl2 * sdelta, 2) + np.power(cl1 * sl2 - sl1 * cl2 * cdelta, 2), 0.5)
    x = sl1 * sl2 + cl1 * cl2 * cdelta
    ad = np.arctan2(y,x)
    dist = ad * rad
    
    return dist

In [3]:
# задаем координаты «центра», сначала широту,
# затем долготу
city_center_coordinates = (55.0415000, 82.9346000)
# создаем переменную – расстояние квартиры от «центра»
data['dist_center'] = calculate_distance(data['Lat'], 
                                         data['Long'],
                                         city_center_coordinates[0], 
                                         city_center_coordinates[1], 
                                         km=True)
# а еще можно воспользоваться готовой функцией haversine()
data['dist_center2'] = data.apply(
    lambda row: haversine((row['Lat'], row['Long']), city_center_coordinates), axis=1)
# смотрим результаты
data.head()

,Rooms_Number,Object_Type,Settlement,District,Street,House_Number,Metro,Metro_m,Space_Total,Lat,Long,Date_Postr,dist_center,dist_center2
0,1,Хрущевка,Новосибирск,Первомайский,Маяковского,24,Речной вокзал,11260,30.5,54.9738,83.0860,1971,12.246147,12.242320
1,1,Типовая,Новосибирск,Первомайский,Звездная,14,Речной вокзал,12130,28.9,54.9551,83.0681,1974,12.841525,12.837513
2,1,Типовая,Новосибирск,Центральный,Лермонтова,36,Сибирская,438,31.9,55.0446,82.9267,1973,0.610227,0.610037
3,2,Хрущевка,Новосибирск,Центральный,Достоевского,20,Сибирская,312,44.7,55.0456,82.9192,1964,1.082215,1.081877
4,2,Улучшенной планировки,Новосибирск,Советский,Демакова,12,Речной вокзал,23340,49.0,54.8634,83.1041,1989,22.575526,22.568473
